<a href="https://colab.research.google.com/github/aishaabduallah/week9/blob/main/Copy_of_RAG_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

## setup

In [ ]:
import os
import pandas as pd
import markdown
from google.colab import userdata

os.environ["LANGSMITH_API_KEY"] = userdata.get('LangChain_api')
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["OPENAI_API_KEY"] = userdata.get('openAI_KEY')

In [ ]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 5.59MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [ ]:
!pip install -q langchain langchain-community  LLMChain
!pip install -q langchain-text-splitters
!pip install -q Chroma
!pip install -q langchain-openai

ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain
  Preparing metadata (setup.py) ... done


# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [ ]:
df = pd.read_csv('/content/Dataset.csv')

In [ ]:
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   المخالفة  104 non-null    object
 1   الغرامة   104 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [ ]:
df.columns

Index(['المخالفة', 'الغرامة'], dtype='object')

In [ ]:
df['المخالفة'].apply(len).max()

141

In [ ]:
df['الغرامة'].apply(len).max()

33

## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [ ]:
data_dir = '/content/violation_data'
os.makedirs(data_dir, exist_ok=True)

In [ ]:
for i in range(len(df)):
  violation = df.iloc[i]['المخالفة']
  fine = df.iloc[i]['الغرامة']

  content = f'# {violation}\n\n'
  content += f'## {fine}\n\n'

  with open(os.path.join(data_dir, f'{i}.md'), 'w') as f:
    f.write(content)

In [ ]:
markdown_texts = []
for filename in os.listdir(data_dir):
  if filename.endswith(".md"):
    with open(os.path.join(data_dir, filename), 'r', encoding='utf-8') as file:
      markdown_content = file.read()
      html_content = markdown.markdown(markdown_content)
      markdown_texts.append(html_content)

In [ ]:
len(df)

104

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=2,
    length_function=len,
    is_separator_regex=False,
)

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name='traffic_violation',
    embedding_function=embeddings,
    persist_directory='./chroma_db',
)

In [ ]:
vector_store.add_texts(markdown_texts)

['4e25ff5c-30a7-4d2e-85e7-0fd1f09f438b',
 '8622f7f0-4c26-481f-8215-e344a43dfb27',
 '196457a9-791f-49f5-aa61-9c32c0bd6128',
 'f1c0afde-2562-46d2-80da-a08103bd86f3',
 'e33e5f0f-4612-4784-bd66-81d43f73d867',
 '2fa8013d-3b44-438a-8280-12daf41e2c26',
 '9c788c3d-3244-472d-9dfd-55fe027abdcb',
 '6ed8f0c9-28fd-4d76-91a2-52043751e9e3',
 '36493493-8e1a-42bf-8395-1ce2d7d2d5ff',
 '4e128474-ef95-4f61-9124-c8ff483f86e4',
 'ce47dffa-1832-4666-8022-47f46ad03983',
 '33cfbfe5-62e7-4219-a8de-b2c26f9c749b',
 '453b3e91-cb4a-4936-ab99-1e5c66c72e47',
 'cc272346-2d45-4866-9244-4faa30760605',
 'a9382bd1-58df-485b-859e-e73f7ea75b66',
 'd210a7b5-6de0-4b57-a62e-867c65b367c1',
 '8a9d485e-43a1-43b8-a6e5-496db545fb02',
 '226c86ed-1368-4551-b4fe-3fee13ccd08d',
 'e4a29460-da30-4cbe-8ae4-7086ae696637',
 '21c90a3b-ce32-412a-87f3-fe4c6be04df6',
 '83ab26b5-5340-45eb-ab6b-4d87ecb2cfe0',
 '744652f4-b683-468d-bb58-2b9cc1ade9e8',
 '6fac41ba-ed34-4077-8804-e9c0b33fbc59',
 '4691eb02-f1f9-430b-b6b9-051c54a5b5dc',
 '6cc65cdd-e1d0-

In [ ]:
results = vector_store.similarity_search(
    'ماهي الغرامة على القيادة بدون رخصة؟',
    k=2,
    )
for res in results:
    print(f"* {res.page_content}")

* <h1>عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة.</h1>
<p>الغرامة المالية 150 - 300 ريال</p>
* <h1>قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>


In [ ]:
def query_from_db(query, db, k=2):
  results = db.similarity_search(
    query,
    k=k,
  )
  return results

In [ ]:
result = query_from_db('ماهي الغرامة على ترك الاطفال بالسيارة؟', vector_store, k=1)
for res in result:
  print(f"* {res.page_content}")

* <h1>ترك الأطفال دون سن العاشرة في المركبة دون مرافق راشد.</h1>
<p>الغرامة المالية 300 - 500 ريال</p>


In [ ]:
result = query_from_db('ماهي مخالفة قطع الاشارة؟', vector_store, k=2)
for res in result:
  print(f"* {res.page_content}")

* <h1>عدم التقيد بإشارات رجل الأمن اليدوية عند تنظيمه للحركة وعدم إعطاء إشارته الأولوية على الإشارات الضوئية.</h1>
<p>الغرامة المالية 500 - 900 ريال</p>
* <h1>عدم تقيد المشاة بالإشارات الخاصة بهم.</h1>
<p>الغرامة المالية 100 - 150 ريال</p>


In [ ]:
result = query_from_db('ما هي مخالفة التجاوز من الاكتاف؟', vector_store, k=3)
for res in result:
  print(f"* {res.page_content}")

* <h1>قيام السائق بتخطي أرتال المركبات أمام إشارات المرور أو نقاط الضبط الأمني باستخدام كتف الطريق أو المسار المخصص للالتفاف.</h1>
<p>الغرامة المالية 150 - 300 ريال</p>
* <h1>التجاوز في المناطق التي يمنع فيها التجاوز، مثل المنعطفات والمرتفعات.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>
* <h1>عدم الالتزام بحدود المسارات المحددة على الطريق.</h1>
<p>الغرامة المالية 300 - 500 ريال</p>


# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [ ]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
جاوب على السوال بناء على السياق فقط:

السياق: {context}

السؤال: {question}

الإجابة:
"""

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=PROMPT_TEMPLATE,
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [ ]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model='gpt-4', temperature=0.1)

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [ ]:
from langchain.chains import LLMChain

MODEL = LLMChain(llm=model,
                 prompt=prompt_template,
                 verbose=True)

<ipython-input-84-02993101f8fa>:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  MODEL = LLMChain(llm=model,


## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [ ]:
def query_rag(query: str, db, k=2):
  results = db.similarity_search_with_score(
    query,
    k=k,
  )
  context_text = "\n".join([res[0].page_content for res in results])
  response = MODEL.run(context=context_text, question=query)
  return response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
result = query_rag('ماهي مخالفة السير بدون لوحات؟', vector_store, k=1)
print(result)

Prompt after formatting:
 
جاوب على السوال بناء على السياق فقط:

السياق: <h1>سير المركبة بلا لوحة خلفية، أو بلا لوحات.</h1>
<p>الغرامة المالية 3000 - 6000 ريال</p>

السؤال: ماهي مخالفة السير بدون لوحات؟

الإجابة:


> Finished chain.
الغرامة المالية لمخالفة السير بدون لوحات تتراوح بين 3000 إلى 6000 ريال.


In [ ]:
result = query_rag('ماهي مخالفة التفحيط؟', vector_store, k=1)
print(result)

Prompt after formatting:
 
جاوب على السوال بناء على السياق فقط:

السياق: <h1>التجمهر في مواقع التفحيط.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>

السؤال: ماهي مخالفة التفحيط؟

الإجابة:


> Finished chain.
المخالفة هي التجمهر في مواقع التفحيط والغرامة المالية لها تتراوح بين 1000 إلى 2000 ريال.


In [ ]:
result = query_rag('ماهي الغرامة على القيادة بدون رخصة؟', vector_store, k=1)
print(result)

<ipython-input-88-30b270fa958f>:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = MODEL.run(context=context_text, question=query)


Prompt after formatting:
 
جاوب على السوال بناء على السياق فقط:

السياق: <h1>عدم حمل رخصة القيادة أو رخصة السير أثناء القيادة.</h1>
<p>الغرامة المالية 150 - 300 ريال</p>

السؤال: ماهي الغرامة على القيادة بدون رخصة؟

الإجابة:


> Finished chain.
الغرامة المالية 150 - 300 ريال.


In [ ]:
result = query_rag('ما هي مخالفه السير في اماكن المشاه؟', vector_store, k=2)
print(result)

Prompt after formatting:
 
جاوب على السوال بناء على السياق فقط:

السياق: <h1>مخالفة تنظيمات السير على الطرق.</h1>
<p>الغرامة المالية 150 - 300 ريال</p>
<h1>التجاوز في المناطق التي يمنع فيها التجاوز، مثل المنعطفات والمرتفعات.</h1>
<p>الغرامة المالية 1000 - 2000 ريال</p>

السؤال: ما هي مخالفه السير في اماكن المشاه؟

الإجابة:


> Finished chain.
السياق لا يوفر معلومات حول مخالفة السير في أماكن المشاه.


In [ ]:
result = query_rag('ماهي غرامة قطع اشارة المرور', vector_store, k=1)
print(result)

Prompt after formatting:
 
جاوب على السوال بناء على السياق فقط:

السياق: <h1>تجاوز إشارة المرور الضوئية أثناء الضوء الأحمر.</h1>
<p>الغرامة المالية 3000 - 6000 ريال</p>

السؤال: ماهي غرامة قطع اشارة المرور

الإجابة:


> Finished chain.
الغرامة المالية تتراوح بين 3000 إلى 6000 ريال.


In [ ]:
result = query_rag('ماهي مخالفة السرعة', vector_store, k=1)
print(result)

Prompt after formatting:
 
جاوب على السوال بناء على السياق فقط:

السياق: <h1>التحرك بالمركبة بسرعة عالية، بحيث تحدث الإطارات صوتاً عالياً.</h1>
<p>الغرامة المالية 150 - 300 ريال</p>

السؤال: ماهي مخالفة السرعة

الإجابة:


> Finished chain.
الغرامة المالية لمخالفة السرعة تتراوح بين 150 إلى 300 ريال.


In [ ]:
result = query_rag('ماهي مخالفة التجاوز من الاكتاف؟', vector_store, k=1)
print(result)

ValueError: Missing some input keys: {'question'}